<a href="https://colab.research.google.com/github/Chiamakac/TRAININGS/blob/main/Finetuning_Igbobert_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Transformers installation
! pip install transformers datasets
# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

# Fine-tuning a pretrained model

In this tutorial, we will show you how to fine-tune a pretrained model from the Transformers library. In TensorFlow,
models can be directly trained using Keras and the `fit` method. In PyTorch, there is no generic training loop so
the 🤗 Transformers library provides an API with the class [Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer) to let you fine-tune or train
a model from scratch easily. Then we will show you how to alternatively write the whole training loop in PyTorch.

Before we can fine-tune a model, we need a dataset. In this tutorial, we will show you how to fine-tune BERT on the
[IMDB dataset](https://www.imdb.com/interfaces/): the task is to classify whether movie reviews are positive or
negative. For examples of other tasks, refer to the [additional-resources](#additional-resources) section!

<a id='data-processing'></a>

## Preparing the datasets

We will use the [🤗 Datasets](https://github.com/huggingface/datasets/) library to download and preprocess the IMDB
datasets. We will go over this part pretty quickly. Since the focus of this tutorial is on training, you should refer
to the 🤗 Datasets [documentation](https://huggingface.co/docs/datasets/) or the [preprocessing](https://huggingface.co/docs/transformers/master/en/preprocessing) tutorial for
more information.

First, we can use the `load_dataset` function to download and cache the dataset:

In [2]:
from datasets import load_dataset

raw_datasets = load_dataset('masakhaner', 'ibo')

Reusing dataset masakhaner (/root/.cache/huggingface/datasets/masakhaner/ibo/1.0.0/e61b24903076a3af7682855beebb820ec64edad0d6787b148c473694592d10b3)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
print(raw_datasets)

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 2235
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 320
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 638
    })
})


This works like the `from_pretrained` method we saw for the models and tokenizers (except the cache directory is
_~/.cache/huggingface/dataset_ by default).

The `raw_datasets` object is a dictionary with three keys: `"train"`, `"test"` and `"validation"`
(which correspond to the three splits of that dataset). We will use the `"train"` split for training and the
`"test"` split for validation.

To preprocess our data, we will need a tokenizer:

In [4]:
from transformers import(
    AutoModelForSequenceClassification,
    AutoModelForTokenClassification,
    AutoConfig,
    AutoTokenizer,
    AutoModel,
    AdamW,
    BertConfig,
    BertForTokenClassification,
    BertTokenizer,
    CamembertConfig,
    CamembertForTokenClassification,
    CamembertTokenizer,
    DistilBertConfig,
    DistilBertForTokenClassification,
    DistilBertTokenizer,
    RobertaConfig,
    RobertaForTokenClassification,
    RobertaTokenizer,
    XLMRobertaConfig,
    XLMRobertaForTokenClassification,
    XLMRobertaTokenizer,
    get_linear_schedule_with_warmup,
    get_constant_schedule_with_warmup,
    DataCollatorWithPadding
)

In [5]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [6]:
MODEL_PATH = "/content/gdrive/MyDrive/IBO_BETA/IgboBert_v2"
checkpoint = "MODEL_PATH"
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model =AutoModelForSequenceClassification.from_pretrained(MODEL_PATH)


Some weights of the model checkpoint at /content/gdrive/MyDrive/IBO_BETA/IgboBert_v2 were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /content/gdrive/MyDrive/IBO_BETA/IgboBert_v2 and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'class

In [32]:
# from tokenizers import ByteLevelBPETokenizer
# from tokenizers.implementations import ByteLevelBPETokenizer
# from tokenizers.processors import BertProcessing

# tokenizer = ByteLevelBPETokenizer(
    # "/content/gdrive/MyDrive/IBO_BETA/IgboBert_v2/vocab.json",
    # "/content/gdrive/MyDrive/IBO_BETA/IgboBert_v2/merges.txt",
# )

# tokenizer._tokenizer.post_processor = BertProcessing(
    # ("</s>", tokenizer.token_to_id("</s>")),
    # ("<s>", tokenizer.token_to_id("<s>")),
# )
# tokenizer.enable_truncation(max_length=512)

In [33]:
# working_dir='/content/gdrive/MyDrive/IBO_BETA/IgboBert_v2'

In [34]:
# from transformers import RobertaTokenizerFast

# tokenizer = RobertaTokenizerFast.from_pretrained(working_dir, max_len=512)

In [ ]:
# from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

As we saw in [preprocessing](https://huggingface.co/docs/transformers/master/en/preprocessing), we can prepare the text inputs for the model with the following command (this is an
example, not a command you can execute):

In [11]:
# class sentences:
  # inputs = tokenizer('sentences', padding="max_length", truncation=True)

This will make all the samples have the maximum length the model can accept (here 512), either by padding or truncating
them.

However, we can instead apply these preprocessing steps to all the splits of our dataset at once by using the
`map` method:

In [7]:
def tokenize_function(examples):
    return tokenizer(examples["id"], padding="max_length", truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Loading cached processed dataset at /root/.cache/huggingface/datasets/masakhaner/ibo/1.0.0/e61b24903076a3af7682855beebb820ec64edad0d6787b148c473694592d10b3/cache-98e557a4f963a7d1.arrow


  0%|          | 0/1 [00:00<?, ?ba/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Loading cached processed dataset at /root/.cache/huggingface/datasets/masakhaner/ibo/1.0.0/e61b24903076a3af7682855beebb820ec64edad0d6787b148c473694592d10b3/cache-6d9393575fbd41cd.arrow


You can learn more about the map method or the other ways to preprocess the data in the 🤗 Datasets [documentation](https://huggingface.co/docs/datasets/).

Next we will generate a small subset of the training and validation set, to enable faster training:

In [8]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(500))
full_train_dataset = tokenized_datasets["train"]
full_eval_dataset = tokenized_datasets["test"]

Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/masakhaner/ibo/1.0.0/e61b24903076a3af7682855beebb820ec64edad0d6787b148c473694592d10b3/cache-0de094a3b04b88fb.arrow
Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/masakhaner/ibo/1.0.0/e61b24903076a3af7682855beebb820ec64edad0d6787b148c473694592d10b3/cache-d07c54bda3d686df.arrow


In all the examples below, we will always use `small_train_dataset` and `small_eval_dataset`. Just replace
them by their _full_ equivalent to train or evaluate on the full dataset.

<a id='trainer'></a>

## Fine-tuning in PyTorch with the Trainer API

In [ ]:
#@title
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/nvBXf7s7vTI?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

Since PyTorch does not provide a training loop, the 🤗 Transformers library provides a [Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer)
API that is optimized for 🤗 Transformers models, with a wide range of training options and with built-in features like
logging, gradient accumulation, and mixed precision.

First, let's define our model:

In [9]:
model =AutoModelForSequenceClassification.from_pretrained(MODEL_PATH,num_labels=2)

Some weights of the model checkpoint at /content/gdrive/MyDrive/IBO_BETA/IgboBert_v2 were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /content/gdrive/MyDrive/IBO_BETA/IgboBert_v2 and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'class

This will issue a warning about some of the pretrained weights not being used and some weights being randomly
initialized. That's because we are throwing away the pretraining head of the BERT model to replace it with a
classification head which is randomly initialized. We will fine-tune this model on our task, transferring the knowledge
of the pretrained model to it (which is why doing this is called transfer learning).

Then, to define our [Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer), we will need to instantiate a
[TrainingArguments](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.TrainingArguments). This class contains all the hyperparameters we can tune for the
[Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer) or the flags to activate the different training options it supports. Let's begin by
using all the defaults, the only thing we then have to provide is a directory in which the checkpoints will be saved:

In [14]:
from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


Then we can instantiate a [Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer) like this:

In [21]:
from transformers import Trainer

trainer = Trainer(model=model, args=training_args, train_dataset=small_train_dataset, eval_dataset=small_eval_dataset)

To fine-tune our model, we just need to call

In [22]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: ner_tags, id, tokens.
***** Running training *****
  Num examples = 1000
  Num Epochs = 5
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 80


ValueError: ignored

which will start a training that you can follow with a progress bar, which should take a couple of minutes to complete
(as long as you have access to a GPU). It won't actually tell you anything useful about how well (or badly) your model
is performing however as by default, there is no evaluation during training, and we didn't tell the
[Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer) to compute any metrics. Let's have a look on how to do that now!

To have the [Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer) compute and report metrics, we need to give it a `compute_metrics`
function that takes predictions and labels (grouped in a namedtuple called [EvalPrediction](https://huggingface.co/docs/transformers/master/en/internal/trainer_utils#transformers.EvalPrediction)) and
return a dictionary with string items (the metric names) and float values (the metric values).

The 🤗 Datasets library provides an easy way to get the common metrics used in NLP with the `load_metric` function.
here we simply use accuracy. Then we define the `compute_metrics` function that just convert logits to predictions
(remember that all 🤗 Transformers models return the logits) and feed them to `compute` method of this metric.

In [ ]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

The compute function needs to receive a tuple (with logits and labels) and has to return a dictionary with string keys
(the name of the metric) and float values. It will be called at the end of each evaluation phase on the whole arrays of
predictions/labels.

To check if this works on practice, let's create a new [Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer) with our fine-tuned model:

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)
trainer.evaluate()

which showed an accuracy of 87.5% in our case.

If you want to fine-tune your model and regularly report the evaluation metrics (for instance at the end of each
epoch), here is how you should define your training arguments:

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer", evaluation_strategy="epoch")

See the documentation of [TrainingArguments](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.TrainingArguments) for more options.


<a id='keras'></a>

<a id='pytorch_native'></a>

## Fine-tuning in native PyTorch

You might need to restart your notebook at this stage to free some memory, or execute the following code:

In [16]:
del model
del pytorch_model
del trainer
torch.cuda.empty_cache()

NameError: ignored

Let's now see how to achieve the same results as in [trainer section](#trainer) in PyTorch. First we need to
define the dataloaders, which we will use to iterate over batches. We just need to apply a bit of post-processing to
our `tokenized_datasets` before doing that to:

- remove the columns corresponding to values the model does not expect (here the `"text"` column)
- rename the column `"label"` to `"labels"` (because the model expect the argument to be named `labels`)
- set the format of the datasets so they return PyTorch Tensors instead of lists.

Our _tokenized_datasets_ has one method for each of those steps:

In [33]:
tokenized_datasets = tokenized_datasets.remove_columns(["id"])
tokenized_datasets = tokenized_datasets.rename_column("ner_tag", "labels")
tokenized_datasets.set_format("torch")

small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(500))

Now that this is done, we can easily define our dataloaders:

In [34]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=8)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=8)

Next, we define our model:

In [35]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH, num_labels=2)

loading configuration file /content/gdrive/MyDrive/IBO_BETA/IgboBert_v2/config.json
Model config RobertaConfig {
  "_name_or_path": "/content/gdrive/MyDrive/IBO_BETA/IgboBert_v2",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.15.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 52000
}

loading weights file /content/gdrive/MyDrive/IBO_BETA/IgboBert_v2/pytorch_model.bin
Some weights of the model checkpoint at /content/gdrive/MyDrive/IBO_BETA/IgboBert_v2 were not used wh

We are almost ready to write our training loop, the only two things are missing are an optimizer and a learning rate
scheduler. The default optimizer used by the [Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer) is [AdamW](https://huggingface.co/docs/transformers/master/en/main_classes/optimizer_schedules#transformers.AdamW):

In [36]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

Finally, the learning rate scheduler used by default is just a linear decay from the maximum value (5e-5 here) to 0:

In [37]:
from transformers import get_scheduler

num_epochs = 5
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

One last thing, we will want to use the GPU if we have access to one (otherwise training might take several hours
instead of a couple of minutes). To do this, we define a `device` we will put our model and our batches on.

In [38]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(52000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

We now are ready to train! To get some sense of when it will be finished, we add a progress bar over our number of
training steps, using the _tqdm_ library.

In [47]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

SyntaxError: ignored

Note that if you are used to freezing the body of your pretrained model (like in computer vision) the above may seem a
bit strange, as we are directly fine-tuning the whole model without taking any precaution. It actually works better
this way for Transformers model (so this is not an oversight on our side). If you're not familiar with what "freezing
the body" of the model means, forget you read this paragraph.

Now to check the results, we need to write the evaluation loop. Like in the [trainer section](#trainer) we will
use a metric from the datasets library. Here we accumulate the predictions at each batch before computing the final
result when the loop is finished.

In [ ]:
metric = load_metric("accuracy")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

<a id='additional-resources'></a>

## Additional resources

To look at more fine-tuning examples you can refer to:

- [🤗 Transformers Examples](https://github.com/huggingface/transformers/tree/master/examples) which includes scripts
  to train on all common NLP tasks in PyTorch and TensorFlow.

- [🤗 Transformers Notebooks](https://huggingface.co/docs/transformers/master/en/notebooks) which contains various notebooks and in particular one per task (look for
  the _how to finetune a model on xxx_).